### Dataframes, Spark SQL e Parquet - practica-- SAMUEL PEDROSA

In [1]:
# Carga o ficheiro purchases.txt desde o sistema de ficheiros localabs

In [2]:
compras = spark.read.csv("purchases.txt",sep="\t")
compras.show()

+----------+-----+--------------+--------------------+------+----------+
|       _c0|  _c1|           _c2|                 _c3|   _c4|       _c5|
+----------+-----+--------------+--------------------+------+----------+
|2012-01-01|09:00|      San Jose|      Men's Clothing|214.05|      Amex|
|2012-01-01|09:00|    Fort Worth|    Women's Clothing|153.57|      Visa|
|2012-01-01|09:00|     San Diego|               Music| 66.08|      Cash|
|2012-01-01|09:00|    Pittsburgh|        Pet Supplies|493.51|  Discover|
|2012-01-01|09:00|         Omaha| Children's Clothing|235.63|MasterCard|
|2012-01-01|09:00|      Stockton|      Men's Clothing|247.18|MasterCard|
|2012-01-01|09:00|        Austin|             Cameras| 379.6|      Visa|
|2012-01-01|09:00|      New York|Consumer Electronics| 296.8|      Cash|
|2012-01-01|09:00|Corpus Christi|                Toys| 25.38|  Discover|
|2012-01-01|09:00|    Fort Worth|                Toys|213.88|      Visa|
|2012-01-01|09:00|     Las Vegas|         Video Gam

In [3]:
compras = compras.toDF('data','hora','tenda','categoria','venda','pago')

In [2]:
# Crea unha táboa sobre o dataframe para poder realizar consultas SQL

In [4]:
compras.createOrReplaceTempView('purchases')

In [3]:
# Mostra os datos do dataframe

In [5]:
compras.show()

+----------+-----+--------------+--------------------+------+----------+
|      data| hora|         tenda|           categoria| venda|      pago|
+----------+-----+--------------+--------------------+------+----------+
|2012-01-01|09:00|      San Jose|      Men's Clothing|214.05|      Amex|
|2012-01-01|09:00|    Fort Worth|    Women's Clothing|153.57|      Visa|
|2012-01-01|09:00|     San Diego|               Music| 66.08|      Cash|
|2012-01-01|09:00|    Pittsburgh|        Pet Supplies|493.51|  Discover|
|2012-01-01|09:00|         Omaha| Children's Clothing|235.63|MasterCard|
|2012-01-01|09:00|      Stockton|      Men's Clothing|247.18|MasterCard|
|2012-01-01|09:00|        Austin|             Cameras| 379.6|      Visa|
|2012-01-01|09:00|      New York|Consumer Electronics| 296.8|      Cash|
|2012-01-01|09:00|Corpus Christi|                Toys| 25.38|  Discover|
|2012-01-01|09:00|    Fort Worth|                Toys|213.88|      Visa|
|2012-01-01|09:00|     Las Vegas|         Video Gam

In [4]:
# Mostra os datos da táboa cunha consulta SQL

In [6]:
spark.sql('SELECT * FROM purchases').show()

+----------+-----+--------------+--------------------+------+----------+
|      data| hora|         tenda|           categoria| venda|      pago|
+----------+-----+--------------+--------------------+------+----------+
|2012-01-01|09:00|      San Jose|      Men's Clothing|214.05|      Amex|
|2012-01-01|09:00|    Fort Worth|    Women's Clothing|153.57|      Visa|
|2012-01-01|09:00|     San Diego|               Music| 66.08|      Cash|
|2012-01-01|09:00|    Pittsburgh|        Pet Supplies|493.51|  Discover|
|2012-01-01|09:00|         Omaha| Children's Clothing|235.63|MasterCard|
|2012-01-01|09:00|      Stockton|      Men's Clothing|247.18|MasterCard|
|2012-01-01|09:00|        Austin|             Cameras| 379.6|      Visa|
|2012-01-01|09:00|      New York|Consumer Electronics| 296.8|      Cash|
|2012-01-01|09:00|Corpus Christi|                Toys| 25.38|  Discover|
|2012-01-01|09:00|    Fort Worth|                Toys|213.88|      Visa|
|2012-01-01|09:00|     Las Vegas|         Video Gam

In [5]:
# Conta o número de vendas por tenda

In [7]:
spark.sql('SELECT COUNT(venda) as Ventas_totais,tenda FROM purchases GROUP BY tenda').show()

+-------------+---------------+
|Ventas_totais|          tenda|
+-------------+---------------+
|        40013|North Las Vegas|
|        40333|        Phoenix|
|        40209|          Omaha|
|        39806|      Anchorage|
|        40086|        Anaheim|
|        40232|     Greensboro|
|        40368|         Dallas|
|        39728|        Oakland|
|        40342|         Laredo|
|        40173|     Scottsdale|
|        40197|    San Antonio|
|        40326|    Bakersfield|
|        40261|        Raleigh|
|        40080|    Chula Vista|
|        40748|   Philadelphia|
|        40099|     Louisville|
|        40254|    Los Angeles|
|        39826|       Chandler|
|        40561|     Sacramento|
|        40321|   Indianapolis|
+-------------+---------------+
only showing top 20 rows



In [6]:
# Fai un total de vendas por categoría

In [8]:
spark.sql('SELECT COUNT(venda) as Ventas_totais,categoria FROM purchases GROUP BY categoria').show()

+-------------+--------------------+
|Ventas_totais|           categoria|
+-------------+--------------------+
|       230469| Children's Clothing|
|       229932|      Sporting Goods|
|       230039|                 CDs|
|       229059|           Computers|
|       229761|Consumer Electronics|
|       229667|   Health and Beauty|
|       229222|        Pet Supplies|
|       230274|                DVDs|
|       230293|                Baby|
|       229749|              Crafts|
|       230050|    Women's Clothing|
|       230237|         Video Games|
|       229787|               Books|
|       230150|               Music|
|       230430|      Men's Clothing|
|       229320|             Cameras|
|       230073|              Garden|
|       229964|                Toys|
+-------------+--------------------+



In [7]:
# Garda o dataframe en HDFS en formato parquet, con particións para as diferentes tendas

In [13]:
#compras.write.parquet('file:///tmp/purchases.parquet')
compras.write.partitionBy("tenda").mode("overwrite").parquet("file:///tmp/output/purchases2.parquet")

In [8]:
# Carga os datos da tenda de San Jose

In [15]:
compras_parquet_partido = spark.read.parquet("file:///tmp/output/purchases2.parquet/tenda=San Jose")

+----------+-----+--------------------+------+----------+
|      data| hora|           categoria| venda|      pago|
+----------+-----+--------------------+------+----------+
|2012-04-16|09:22|                 CDs| 94.52|  Discover|
|2012-01-01|09:00|      Men's Clothing|214.05|      Amex|
|2012-04-16|09:29|Consumer Electronics|381.52|MasterCard|
|2012-01-01|09:00|    Women's Clothing|215.82|      Cash|
|2012-04-16|09:36|      Sporting Goods|145.74|      Amex|
|2012-01-01|09:09|                Toys|337.71|      Cash|
|2012-04-16|09:37|                DVDs|212.34|      Amex|
|2012-01-01|09:17|              Garden|192.82|      Cash|
|2012-04-16|09:51| Children's Clothing|105.58|MasterCard|
|2012-01-01|09:19|             Cameras| 95.81|      Cash|
|2012-04-16|09:53|           Computers|307.14|      Cash|
|2012-01-01|09:19|        Pet Supplies|253.33|  Discover|
|2012-04-16|09:55|                Baby| 141.5|      Visa|
|2012-01-01|09:20|           Computers| 160.6|      Amex|
|2012-04-16|09

In [9]:
# Mostra os datos do dataframe

In [16]:
compras_parquet_partido.show()

+----------+-----+--------------------+------+----------+
|      data| hora|           categoria| venda|      pago|
+----------+-----+--------------------+------+----------+
|2012-04-16|09:22|                 CDs| 94.52|  Discover|
|2012-01-01|09:00|      Men's Clothing|214.05|      Amex|
|2012-04-16|09:29|Consumer Electronics|381.52|MasterCard|
|2012-01-01|09:00|    Women's Clothing|215.82|      Cash|
|2012-04-16|09:36|      Sporting Goods|145.74|      Amex|
|2012-01-01|09:09|                Toys|337.71|      Cash|
|2012-04-16|09:37|                DVDs|212.34|      Amex|
|2012-01-01|09:17|              Garden|192.82|      Cash|
|2012-04-16|09:51| Children's Clothing|105.58|MasterCard|
|2012-01-01|09:19|             Cameras| 95.81|      Cash|
|2012-04-16|09:53|           Computers|307.14|      Cash|
|2012-01-01|09:19|        Pet Supplies|253.33|  Discover|
|2012-04-16|09:55|                Baby| 141.5|      Visa|
|2012-01-01|09:20|           Computers| 160.6|      Amex|
|2012-04-16|09

In [10]:
# Crea unha táboa sobre o dataframe dos datos de San Jose

In [17]:
compras_parquet_partido.createOrReplaceTempView('compras_San_Jose')

In [11]:
# Mostra os datos da táboa

In [18]:
spark.sql('SELECT * FROM compras_San_Jose').show()

+----------+-----+--------------------+------+----------+
|      data| hora|           categoria| venda|      pago|
+----------+-----+--------------------+------+----------+
|2012-04-16|09:22|                 CDs| 94.52|  Discover|
|2012-01-01|09:00|      Men's Clothing|214.05|      Amex|
|2012-04-16|09:29|Consumer Electronics|381.52|MasterCard|
|2012-01-01|09:00|    Women's Clothing|215.82|      Cash|
|2012-04-16|09:36|      Sporting Goods|145.74|      Amex|
|2012-01-01|09:09|                Toys|337.71|      Cash|
|2012-04-16|09:37|                DVDs|212.34|      Amex|
|2012-01-01|09:17|              Garden|192.82|      Cash|
|2012-04-16|09:51| Children's Clothing|105.58|MasterCard|
|2012-01-01|09:19|             Cameras| 95.81|      Cash|
|2012-04-16|09:53|           Computers|307.14|      Cash|
|2012-01-01|09:19|        Pet Supplies|253.33|  Discover|
|2012-04-16|09:55|                Baby| 141.5|      Visa|
|2012-01-01|09:20|           Computers| 160.6|      Amex|
|2012-04-16|09

In [12]:
# Consulta o total de vendas de San José por tipo de elementos (categoría)

In [19]:
spark.sql('SELECT count(venda) as total_vendas,categoria FROM compras_San_Jose GROUP BY categoria').show()

+------------+--------------------+
|total_vendas|           categoria|
+------------+--------------------+
|        2173| Children's Clothing|
|        2180|      Sporting Goods|
|        2325|                 CDs|
|        2264|           Computers|
|        2240|Consumer Electronics|
|        2233|   Health and Beauty|
|        2179|        Pet Supplies|
|        2239|                DVDs|
|        2268|                Baby|
|        2208|              Crafts|
|        2199|    Women's Clothing|
|        2298|         Video Games|
|        2107|               Books|
|        2220|               Music|
|        2229|      Men's Clothing|
|        2215|              Garden|
|        2143|             Cameras|
|        2178|                Toys|
+------------+--------------------+

